In [3]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
from shapely.geometry import Point, Polygon
import geopandas as gpd

pd.set_option("display.precision", 3)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', 15)

In [4]:
df_bal8_azr_vshclp2 = pd.read_csv('C:\jupyter\SPP\inputoutput\df_bal8_azr_vshclp2.csv').drop('Unnamed: 0', axis=1)

df_bal8_azr_vshclp2 = df_bal8_azr_vshclp2[~df_bal8_azr_vshclp2.well.isin(['E20Y', 'E10Z', 'J13X'])]
well_phit_flag8 = df_bal8_azr_vshclp2[df_bal8_azr_vshclp2.phit_flag==1].groupby('well')['phit_flag'].apply(lambda x: x.iloc[0]).reset_index().well.unique()

In [ ]:
x_max = df_bal8_azr_vshclp2.X_traj.max()
x_min = df_bal8_azr_vshclp2.X_traj.min()
y_max = df_bal8_azr_vshclp2.Y_traj.max()
y_min = df_bal8_azr_vshclp2.Y_traj.min()
print(x_min, x_max, y_min, y_max)
print(int(x_max-x_min), int(y_max-y_min))
x_mesh = np.arange(x_min-300, x_max+300, 356)
y_mesh = np.arange(y_min-300, y_max+300, 200)
X, Y = np.meshgrid(x_mesh, y_mesh)
wells = df_bal8_azr_vshclp2.groupby('well')[['Xmean', 'Ymean']].apply(lambda x: x.iloc[0]).reset_index()
plt.subplots(figsize=(7, 5))
plt.scatter(X, Y, s=1, c='r', marker='o')
plt.scatter(wells.Xmean, wells.Ymean, s=10, c='b', marker='o')
print(x_mesh.shape, y_mesh.shape)

In [6]:
X_flat = X.ravel()
Y_flat = Y.ravel()
df = pd.DataFrame({'X': X_flat, 'Y': Y_flat})
df = df.reset_index()
df['well'] = 'well_' + df['index'].astype(str)
df = df.drop('index', axis=1)

In [70]:
geometry_fld = [Point(xy) for xy in zip(df_bal8_azr_vshclp2['Xmean'], df_bal8_azr_vshclp2['Ymean'])]
geometry_fld = gpd.GeoDataFrame(geometry_fld, geometry=geometry_fld)
buffers_fld = geometry_fld.buffer(250)
field_polygon = gpd.GeoDataFrame(geometry=[buffers_fld.unary_union.convex_hull])

In [64]:
geometry_wells = [Point(xy) for xy in zip(df['X'], df['Y'])]
geometry_wells = gpd.GeoDataFrame(geometry_wells, geometry=geometry_wells)

In [72]:
intersection_points = gpd.sjoin(geometry_wells, field_polygon, op='within')

In [ ]:
fig, ax = plt.subplots(figsize=(15, 7))
field_polygon.plot(ax=ax, color='green', alpha=0.5)
x.scatter(intersection_points.geometry.x, intersection_points.geometry.y, s=1, c='r', marker='o')
ax.scatter(wells.Xmean, wells.Ymean, s=10, c='b', marker='+')